In [1]:
import pandas as pd
df = pd.read_csv('data/WA_Fn-UseC_-Telco-Customer-Churn.csv')

# 1. 결측치 처리

### 결측치 유무 확인

In [2]:
for col in df.select_dtypes(include='object').columns:
    has_blank_or_nan = df[col].isna().sum() + (df[col] == ' ').sum()
    print(f"{col}: {has_blank_or_nan} 개의 공백 또는 NaN")
    

customerID: 0 개의 공백 또는 NaN
gender: 0 개의 공백 또는 NaN
Partner: 0 개의 공백 또는 NaN
Dependents: 0 개의 공백 또는 NaN
PhoneService: 0 개의 공백 또는 NaN
MultipleLines: 0 개의 공백 또는 NaN
InternetService: 0 개의 공백 또는 NaN
OnlineSecurity: 0 개의 공백 또는 NaN
OnlineBackup: 0 개의 공백 또는 NaN
DeviceProtection: 0 개의 공백 또는 NaN
TechSupport: 0 개의 공백 또는 NaN
StreamingTV: 0 개의 공백 또는 NaN
StreamingMovies: 0 개의 공백 또는 NaN
Contract: 0 개의 공백 또는 NaN
PaperlessBilling: 0 개의 공백 또는 NaN
PaymentMethod: 0 개의 공백 또는 NaN
TotalCharges: 11 개의 공백 또는 NaN
Churn: 0 개의 공백 또는 NaN


### TotalCharges의 결측치 채우기: 중앙값

In [3]:
# 'TotalCharges'를 숫자로 변환 (float64), 변환 불가능한 값은 NaN으로 처리
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# 결측치 채우기 (중앙값 사용)
median_value = df['TotalCharges'].median()
df['TotalCharges'] = df['TotalCharges'].fillna(median_value)  # 결측치 채우기

# 결과 확인
print(df['TotalCharges'].dtype)         # 데이터 타입 확인
print(df['TotalCharges'].isna().sum())  # 결측치 확인

float64
0


# 2. 이상치 처리하기

In [4]:
def find_outliers_and_replace(df, whis=1.5):
    """
    주어진 데이터프레임에서 숫자형 열들에 대해 IQR 방식을 통해 이상치를 찾아
    이상치 값을 최소 또는 최대 허용 범위 값으로 대체하는 함수.
    이진 변수는 이상치 처리에서 제외.
    
    Parameters:
    - df: 데이터프레임
    - whis: IQR의 배수로, 기본값은 1.5
    
    Returns:
    - 이상치가 대체된 데이터프레임
    """
    # 복사본 생성
    df_copy = df.copy()
    
    # 숫자형 열만 선택
    numeric_columns = df_copy.select_dtypes(include='number').columns
    
    for column_name in numeric_columns:
        # 이진 변수인지 확인 (고유 값이 2개만 있는 경우)
        unique_values = df_copy[column_name].nunique()
        if unique_values == 2:
            # 이진 변수인 경우 이상치 처리를 생략
            continue

        # 1사분위수(Q1)와 3사분위수(Q3) 계산
        q1, q3 = df_copy[column_name].quantile([0.25, 0.75])
        iqr = q3 - q1

        # 하한과 상한 계산
        lower_bound = q1 - iqr * whis
        upper_bound = q3 + iqr * whis

        # 이상치를 하한과 상한으로 대체
        df_copy[column_name] = df_copy[column_name].apply(
            lambda x: lower_bound if x < lower_bound else (upper_bound if x > upper_bound else x)
        )
    
    return df_copy

# 3. 인코딩(One Hot encoding) 처리

In [9]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

# 1. Label Encoding (이진 변수)
label_cols = ['gender', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']
label_encoder = LabelEncoder()

for col in label_cols:
    df[col] = label_encoder.fit_transform(df[col])

# 2. One-Hot Encoding (다중 범주 변수)
df = pd.get_dummies(df, columns=['InternetService', 'Contract', 'PaymentMethod', 'MultipleLines',
                                 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
                                 'StreamingTV', 'StreamingMovies'])

In [10]:
# Column 확인하기
for idx, col in enumerate(df.columns):
    print(f"Index: {idx}, Column Name: {col}")

Index: 0, Column Name: customerID
Index: 1, Column Name: gender
Index: 2, Column Name: SeniorCitizen
Index: 3, Column Name: Partner
Index: 4, Column Name: Dependents
Index: 5, Column Name: tenure
Index: 6, Column Name: PhoneService
Index: 7, Column Name: PaperlessBilling
Index: 8, Column Name: MonthlyCharges
Index: 9, Column Name: TotalCharges
Index: 10, Column Name: Churn
Index: 11, Column Name: InternetService_DSL
Index: 12, Column Name: InternetService_Fiber optic
Index: 13, Column Name: InternetService_No
Index: 14, Column Name: Contract_Month-to-month
Index: 15, Column Name: Contract_One year
Index: 16, Column Name: Contract_Two year
Index: 17, Column Name: PaymentMethod_Bank transfer (automatic)
Index: 18, Column Name: PaymentMethod_Credit card (automatic)
Index: 19, Column Name: PaymentMethod_Electronic check
Index: 20, Column Name: PaymentMethod_Mailed check
Index: 21, Column Name: MultipleLines_No
Index: 22, Column Name: MultipleLines_No phone service
Index: 23, Column Name: M

In [ ]:
import os

# data 폴더 생성 (존재하지 않으면 생성)
os.makedirs('data', exist_ok=True)

# CSV 파일 저장
df.to_csv('data/df_encoded.csv', index=False)
print("전처리된 데이터가 'data/df_encoded.csv' 파일로 저장되었습니다.")

전처리된 데이터가 'df_encoded.csv' 파일로 저장되었습니다.
